In [1]:
#@title codes to mount your google drive folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NLP_Test_Distilling/

Mounted at /content/drive
/content/drive/MyDrive/NLP_Test_Distilling


In [2]:
!pip install transformers
!pip install transformers[torch]
!pip install datasets
!pip install sentencepiece
!pip install accelerate -U
!pip install tqdm
!pip install wandb
!sudo apt-get install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1

In [3]:
import torch

from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollator, T5ForConditionalGeneration, T5TokenizerFast

from tqdm import tqdm

from typing import Dict, List, Optional

import dataclasses
from dataclasses import dataclass, field

import logging
import os
import sys

import numpy as np
import torch

from huggingface_hub import notebook_login

from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)

from google.colab import files

In [4]:
notebook_login()

In [5]:
import json

with open('SLM_inputs_train.jsonl', 'r') as json_file:
  json_list = list(json_file)

# with open('SLM_inputs_with_prompts_train.jsonl', 'w') as output_file:
l = []
for json_str in json_list:
  result = json.loads(json_str)
  result['question'] = "Generate rationale and answer: " + result['question']
  l.append(result)
print(l[0])

with open('output.jsonl', 'w') as outfile:
  for entry in l:
    json.dump(entry, outfile)
    outfile.write('\n')



{'question': 'Generate rationale and answer: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'full_answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'exact_answer': '72', 'rationale': '<pad> Natalia sold 48 / 2 = 24 clips in May. So in April and May, Natalia sold 48 + 24 = 96 clips. The answer is 96.</s>', 'predicted_answer': '96.</s>'}


In [6]:
from datasets import load_dataset, load_metric
dataset = load_dataset("json", data_files= "output.jsonl")
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0828e2d3b8572535/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer'],
        num_rows: 7473
    })
})


# 100% dataset

In [7]:
datasets_train_test = dataset['train'].train_test_split(test_size = 1500)
datasets_train_validation = datasets_train_test['train'].train_test_split(test_size = 1500)
dataset['train'] =datasets_train_validation['train']
dataset['validation'] = datasets_train_validation['test']
dataset['test']=datasets_train_test['test']

dataset['train'] =dataset['train'].shuffle().select(range(len(dataset['train'])))
dataset['validation'] =dataset['validation'].shuffle().select(range(len(dataset['validation'])))
dataset['test'] =dataset['test'].shuffle().select(range(len(dataset['test'])))

In [8]:
checkpoint = "google/flan-t5-small"
model_100 = T5ForConditionalGeneration.from_pretrained(checkpoint).cuda()
tokenizer_100 = T5TokenizerFast.from_pretrained(checkpoint)

In [9]:
max_input_length =  512
max_target_length = 64

In [10]:
# tokenize the examples
def convert_to_features_100(example_batch):

    input_encodings = tokenizer_100.batch_encode_plus(example_batch['question'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=False,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer_100.batch_encode_plus(example_batch['rationale'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=False,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

def add_eos_examples(example):
  example['question'] = example['question']
  example['rationale'] = example['rationale']
  return example


def add_special_tokens(example):
  example['rationale'] = example['rationale']
  return example

In [11]:
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features_100,  batched=True)

Map:   0%|          | 0/4473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4473 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset["train"][0]["question"]

In [13]:
tokenized_dataset["train"][0]["rationale"]

'<pad> Rex is taking 2 * 2 = 4 hour-long lessons. He has 40 / 4 = 10 hour-long lessons left. He has 6 * 10 = 60 hours left to take his test. He needs to take 60 - 40 = 20 more hours to reach his goal. The answer is 20.</s>'

In [14]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 4473
    })
    validation: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 1500
    })
})


In [15]:
tokenized_dataset = tokenized_dataset.remove_columns(
    ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer',]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [16]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

In [17]:
# This dataclass implementation is taken from Suraj Patil: https://github.com/patil-suraj/question_generation
@dataclass
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
    """
    Take a list of samples from a Dataset and collate them into a batch.
    Returns:
    A dictionary of tensors
    """

    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': lm_labels,
        'decoder_attention_mask': decoder_attention_mask
    }

In [18]:
training_args = TrainingArguments(output_dir="models/",
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  gradient_accumulation_steps=16,
                                  learning_rate=1e-4,
                                  num_train_epochs=7,
                                  logging_steps=100,
                                  run_name="end2end-questions-generation",
                                  evaluation_strategy="steps",
                                  save_steps=500,
                                  # report_to="wandb",
                                  push_to_hub=False,
                                  push_to_hub_model_id="flan-t5-base-samsum")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1592: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case wjlim2022/flan-t5-base-samsum).
  warnings.warn(


In [ ]:
# logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = Trainer(
    model=model_100,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.136500,0.818176
200,1.060200,0.818176
300,1.058600,0.818176
400,1.061000,0.818176


TrainOutput(global_step=483, training_loss=1.0758084283358809, metrics={'train_runtime': 1457.6011, 'train_samples_per_second': 21.481, 'train_steps_per_second': 0.331, 'total_flos': 5742901252325376.0, 'train_loss': 1.0758084283358809, 'epoch': 6.91})

In [ ]:
def run_model_100(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = input_string
  input_ids = tokenizer_100.encode(input_string, return_tensors="pt").cuda()
  res = model_100.generate(input_ids, **generator_args)
  output = tokenizer_100.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output

In [ ]:
run_model_100(
    "Rhonda, Sally, and Diane are members of their school's track team.  The three of them run the 600-meter relay race together. Rhonda runs the first 200 meters of the race, Sally runs the second 200 meters of the race, and Diane runs the final 200 meters of the race. Rhonda can run 200 meters in 24 seconds.  Sally takes two seconds longer to run the same distance, while Diane can run 200 meters three seconds faster than Rhonda.  How many seconds does it take for the three of them to run the 600-meter relay race?"
)

[['Sally runs the first 200 meters of the race, so she runs 200 / 2 = 200 meters. Sally takes 2 seconds longer than Rhonda, so he takes 2 + 2 = 3 seconds to run the race. The three of them run 200 meters, so it takes 3 + 3 = 10 seconds. The answer is 10.']]

In [ ]:
output_100_percent_trained = []
for line_idx in range(len(l)):
  if line_idx %100 == 0:
    print(line_idx)
  line = l[line_idx]
  qns = line['question']
  op = run_model_100(qns)
  output_100_percent_trained.append({'output': op, 'actual_explanation': line['full_answer'], 'actual_answer': line['exact_answer']})
print(output_100_percent_trained[0:10])

0
100


Exception ignored in: <function _xla_gc_callback at 0x7f865bf8cee0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:7                                                                              │
│ in run_model:11                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1611 in generate        │
│                                                                                                  │
│   1608 │   │   │   │   **model_kwargs,                                                           │
│   1609 │   │   │   )                                                                             │
│   1610 │   │   │   # 13. run beam search                                                         │
│ ❱ 1611 │   │   │   return self.beam_search(                                                      │
│   1612 │   │   │   │   input_ids,                                                                │
│   1613 │   │   │   │   beam_scorer,                                                              │
│   1614 │   │   │   │   logits_processor=logits_processor,                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2909 in beam_search     │
│                                                                                                  │
│   2906 │   │   │                                                                                 │
│   2907 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2908 │   │   │                                                                                 │
│ ❱ 2909 │   │   │   outputs = self(                                                               │
│   2910 │   │   │   │   **model_inputs,                                                           │
│   2911 │   │   │   │   return_dict=True,                                                         │
│   2912 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

In [ ]:
with open('result_100_percent_trained.jsonl', 'w') as outfile:
  for entry in output_100_percent_trained:
    json.dump(entry, outfile)
    outfile.write('\n')

# 75% of the data

In [ ]:
checkpoint = "google/flan-t5-small"
model_75 = T5ForConditionalGeneration.from_pretrained(checkpoint).cuda()
tokenizer_75 = T5TokenizerFast.from_pretrained(checkpoint)

def run_model_75(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = input_string
  input_ids = tokenizer_75.encode(input_string, return_tensors="pt").cuda()
  res = model_75.generate(input_ids, **generator_args)
  output = tokenizer_75.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output


# tokenize the examples
def convert_to_features_75(example_batch):

    input_encodings = tokenizer_75.batch_encode_plus(example_batch['question'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=False,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer_75.batch_encode_plus(example_batch['rationale'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=False,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

dataset = load_dataset("json", data_files= "output.jsonl")
print(dataset)
datasets_train_test = dataset['train'].train_test_split(test_size = 1500)
datasets_train_validation = datasets_train_test['train'].train_test_split(test_size = 1500)
dataset['train'] =datasets_train_validation['train']
dataset['validation'] = datasets_train_validation['test']
dataset['test']=datasets_train_test['test']

percent = 0.75
dataset['train'] =dataset['train'].shuffle().select(range(int(len(dataset['train']) * percent)))
dataset['validation'] =dataset['validation'].shuffle().select(range(int(len(dataset['validation']) * percent)))
dataset['test'] =dataset['test'].shuffle().select(range(int(len(dataset['test']) * percent)))
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features_75,  batched=True)
print(tokenized_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer',]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)
torch.save(train_dataset, 'train_data_75p.pt')
torch.save(valid_dataset, 'valid_data_75p.pt')
# logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = Trainer(
    model=model_75,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()

output_75_percent_trained = []
for line_idx in range(len(l)):
  if line_idx %100 == 0:
    print(line_idx)
  line = l[line_idx]
  qns = line['question']
  op = run_model_75(qns)
  output_75_percent_trained.append({'output': op, 'actual_explanation': line['full_answer'], 'actual_answer': line['exact_answer']})
print(output_75_percent_trained[0:10])

with open('result_75_percent_trained.jsonl', 'w') as outfile:
  for entry in output_75_percent_trained:
    json.dump(entry, outfile)
    outfile.write('\n')

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer'],
        num_rows: 7473
    })
})


Map:   0%|          | 0/3354 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
100,1.132100,0.810693
200,0.991800,0.785337
300,0.957300,0.773685


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
[{'output': [['Natalia sold 48 / 2 = 48 clips in April. The answer is 48.']], 'actual_explanation': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'actual_answer': '72'}, {'output': [['She did 50 minutes of babysitting for a total of 50 + 50 = 120 minutes. So, she earned 120 / 120 = 10 minutes. The answer is 10.']], 'actual_explanation': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10', 'actual_answer': '10'}, {'output': [['Betty has $100 x 2 = $1200. Her parents gave her $15

# 50% of data

In [ ]:
checkpoint = "google/flan-t5-small"
model_50 = T5ForConditionalGeneration.from_pretrained(checkpoint).cuda()
tokenizer_50 = T5TokenizerFast.from_pretrained(checkpoint)

def run_model_50(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = input_string
  input_ids = tokenizer_50.encode(input_string, return_tensors="pt").cuda()
  res = model_50.generate(input_ids, **generator_args)
  output = tokenizer_50.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output


# tokenize the examples
def convert_to_features_50(example_batch):

    input_encodings = tokenizer_50.batch_encode_plus(example_batch['question'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=False,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer_50.batch_encode_plus(example_batch['rationale'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=False,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings


dataset = load_dataset("json", data_files= "output.jsonl")
print(dataset)
datasets_train_test = dataset['train'].train_test_split(test_size = 1500)
datasets_train_validation = datasets_train_test['train'].train_test_split(test_size = 1500)
dataset['train'] =datasets_train_validation['train']
dataset['validation'] = datasets_train_validation['test']
dataset['test']=datasets_train_test['test']

percent = 0.5
dataset['train'] =dataset['train'].shuffle().select(range(int(len(dataset['train']) * percent)))
dataset['validation'] =dataset['validation'].shuffle().select(range(int(len(dataset['validation']) * percent)))
dataset['test'] =dataset['test'].shuffle().select(range(int(len(dataset['test']) * percent)))
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features_50,  batched=True)
print(tokenized_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer',]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)
torch.save(train_dataset, 'train_data_75p.pt')
torch.save(valid_dataset, 'valid_data_75p.pt')
# logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = Trainer(
    model=model_50,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()

trainer.evaluate()

output_50_percent_trained = []
for line_idx in range(len(l)):
  if line_idx %100 == 0:
    print(line_idx)
  line = l[line_idx]
  qns = line['question']
  op = run_model_50(qns)
  output_50_percent_trained.append({'output': op, 'actual_explanation': line['full_answer'], 'actual_answer': line['exact_answer']})
print(output_50_percent_trained[0:10])

with open('result_50_percent_trained.jsonl', 'w') as outfile:
  for entry in output_50_percent_trained:
    json.dump(entry, outfile)
    outfile.write('\n')

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer'],
        num_rows: 7473
    })
})


Map:   0%|          | 0/2236 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 2236
    })
    validation: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 750
    })
    test: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 750
    })
})


Step,Training Loss,Validation Loss
100,1.164500,0.849214
200,1.110700,0.849214


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
[{'output': [['In April and May, Natalia sold 48 * 2 = 480 clips. The answer is 480.']], 'actual_explanation': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'actual_answer': '72'}, {'output': [['The babysitting cost 12 * 2 = 40 minutes. The total cost is 40 - 40 = $120. The answer is 10.']], 'actual_explanation': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10', 'actual_answer': '10'}, {'output': [['Betty has only half of the money she needs, so she needs to save $100 / 2 = 

# 25% of data

In [ ]:
checkpoint = "google/flan-t5-small"
model_25 = T5ForConditionalGeneration.from_pretrained(checkpoint).cuda()
tokenizer_25 = T5TokenizerFast.from_pretrained(checkpoint)

def run_model_25(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = input_string
  input_ids = tokenizer_25.encode(input_string, return_tensors="pt").cuda()
  res = model_25.generate(input_ids, **generator_args)
  output = tokenizer_25.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output

# tokenize the examples
def convert_to_features_25(example_batch):

    input_encodings = tokenizer_25.batch_encode_plus(example_batch['question'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=False,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer_25.batch_encode_plus(example_batch['rationale'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=False,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

dataset = load_dataset("json", data_files= "output.jsonl")
print(dataset)
datasets_train_test = dataset['train'].train_test_split(test_size = 1500)
datasets_train_validation = datasets_train_test['train'].train_test_split(test_size = 1500)
dataset['train'] =datasets_train_validation['train']
dataset['validation'] = datasets_train_validation['test']
dataset['test']=datasets_train_test['test']

percent = 0.25
dataset['train'] =dataset['train'].shuffle().select(range(int(len(dataset['train']) * percent)))
dataset['validation'] =dataset['validation'].shuffle().select(range(int(len(dataset['validation']) * percent)))
dataset['test'] =dataset['test'].shuffle().select(range(int(len(dataset['test']) * percent)))
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features_25,  batched=True)
print(tokenized_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer',]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)
torch.save(train_dataset, 'train_data_75p.pt')
torch.save(valid_dataset, 'valid_data_75p.pt')
# logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = Trainer(
    model=model_25,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()
trainer.evaluate(eval_dataset = dataset['test'])

# output_25_percent_trained = []
# for line_idx in range(len(l)):
#   if line_idx %100 == 0:
#     print(line_idx)
#   line = l[line_idx]
#   qns = line['question']
#   op = run_model_25(qns)
#   output_25_percent_trained.append({'output': op, 'actual_explanation': line['full_answer'], 'actual_answer': line['exact_answer']})
# print(output_25_percent_trained[0:10])

# with open('result_25_percent_trained.jsonl', 'w') as outfile:
#   for entry in output_25_percent_trained:
#     json.dump(entry, outfile)
#     outfile.write('\n')

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer'],
        num_rows: 7473
    })
})


Map:   0%|          | 0/1118 [00:00<?, ? examples/s]

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 1118
    })
    validation: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 375
    })
    test: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 375
    })
})


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/datasets/formatting/formatting.py:100: RuntimeWarning: divide by zero encountered in remainder
  return table.fast_gather(key % table.num_rows)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 84>:84                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3053 in evaluate                 │
│                                                                                                  │
│   3050 │   │   start_time = time.time()                                                          │
│   3051 │   │                                                                                     │
│   3052 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3053 │   │   output = eval_loop(                                                               │
│   3054 │   │   │   eval_dataloader,                                                              │
│   3055 │   │   │   description="Evaluation",                                                     │
│   3056 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3235 in evaluation_loop          │
│                                                                                                  │
│   3232 │   │                                                                                     │
│   3233 │   │   observed_num_examples = 0                                                         │
│   3234 │   │   # Main evaluation loop                                                            │
│ ❱ 3235 │   │   for step, inputs in enumerate(dataloader):                                        │
│   3236 │   │   │   # Update the observed num examples                                            │
│   3237 │   │   │   observed_batch_size = find_batch_size(inputs)                                 │
│   3238 │   │   │   if observed_batch_size is not None:                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:677 in _next_data         │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                        

# 100% of data

In [ ]:
checkpoint = "google/flan-t5-small"
model_100 = T5ForConditionalGeneration.from_pretrained(checkpoint).cuda()
tokenizer_100 = T5TokenizerFast.from_pretrained(checkpoint)

def run_model_100(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = input_string
  input_ids = tokenizer_100.encode(input_string, return_tensors="pt").cuda()
  res = model_100.generate(input_ids, **generator_args)
  output = tokenizer_100.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output

# tokenize the examples
def convert_to_features_100(example_batch):

    input_encodings = tokenizer_100.batch_encode_plus(example_batch['question'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=False,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer_100.batch_encode_plus(example_batch['rationale'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=False,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

dataset = load_dataset("json", data_files= "output.jsonl")
print(dataset)
datasets_train_test = dataset['train'].train_test_split(test_size = 1500)
datasets_train_validation = datasets_train_test['train'].train_test_split(test_size = 1500)
dataset['train'] =datasets_train_validation['train']
dataset['validation'] = datasets_train_validation['test']
dataset['test']=datasets_train_test['test']

percent = 1
dataset['train'] =dataset['train'].shuffle().select(range(int(len(dataset['train']) * percent)))
dataset['validation'] =dataset['validation'].shuffle().select(range(int(len(dataset['validation']) * percent)))
dataset['test'] =dataset['test'].shuffle().select(range(int(len(dataset['test']) * percent)))
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features_25,  batched=True)
print(tokenized_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer',]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)
torch.save(train_dataset, 'train_data_75p.pt')
torch.save(valid_dataset, 'valid_data_75p.pt')
# logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = Trainer(
    model=model_100,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()

output_100_percent_trained = []
for line_idx in range(len(l)):
  if line_idx %100 == 0:
    print(line_idx)
  line = l[line_idx]
  qns = line['question']
  op = run_model_100(qns)
  output_100_percent_trained.append({'output': op, 'actual_explanation': line['full_answer'], 'actual_answer': line['exact_answer']})
print(output_100_percent_trained[0:10])

with open('result_100_percent_trained.jsonl', 'w') as outfile:
  for entry in output_100_percent_trained:
    json.dump(entry, outfile)
    outfile.write('\n')

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer'],
        num_rows: 7473
    })
})


Map:   0%|          | 0/4473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 57>:57                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'convert_to_features_25' is not defined

# Fine tune on 100% of the ground truth human-annotated labels

In [ ]:
import json

with open('SLM_inputs_train.jsonl', 'r') as json_file:
  json_list = list(json_file)

# with open('SLM_inputs_with_prompts_train.jsonl', 'w') as output_file:
l = []
for json_str in json_list:
  result = json.loads(json_str)
  result['question'] = "Generate rationale and answer: " + result['question']
  l.append(result)
print(l[0])

with open('output_actual.jsonl', 'w') as outfile:
  for entry in l:
    json.dump(entry, outfile)
    outfile.write('\n')



{'question': 'Generate rationale and answer: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'full_answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'exact_answer': '72', 'rationale': '<pad> Natalia sold 48 / 2 = 24 clips in May. So in April and May, Natalia sold 48 + 24 = 96 clips. The answer is 96.</s>', 'predicted_answer': '96.</s>'}


In [ ]:
from datasets import load_dataset, load_metric
dataset = load_dataset("json", data_files= "output_actual.jsonl")
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-fe145959948f843d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer'],
        num_rows: 7473
    })
})


In [ ]:
datasets_train_test = dataset['train'].train_test_split(test_size = 1500)
datasets_train_validation = datasets_train_test['train'].train_test_split(test_size = 1500)
dataset['train'] =datasets_train_validation['train']
dataset['validation'] = datasets_train_validation['test']
dataset['test']=datasets_train_test['test']

dataset['train'] =dataset['train'].shuffle().select(range(len(dataset['train'])))
dataset['validation'] =dataset['validation'].shuffle().select(range(len(dataset['validation'])))
dataset['test'] =dataset['test'].shuffle().select(range(len(dataset['test'])))

In [20]:
checkpoint = "google/flan-t5-small"
model_finetune = T5ForConditionalGeneration.from_pretrained(checkpoint).cuda()
tokenizer_finetune = T5TokenizerFast.from_pretrained(checkpoint)

def run_model_fine_tune(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = input_string
  input_ids = tokenizer_finetune.encode(input_string, return_tensors="pt").cuda()
  res = model_finetune.generate(input_ids, **generator_args)
  output = tokenizer_finetune.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output

# tokenize the examples
def convert_to_features_finetune(example_batch):

    input_encodings = tokenizer_finetune.batch_encode_plus(example_batch['question'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=False,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer_finetune.batch_encode_plus(example_batch['full_answer'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=False,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

dataset = load_dataset("json", data_files= "output.jsonl")
print(dataset)
datasets_train_test = dataset['train'].train_test_split(test_size = 1500)
datasets_train_validation = datasets_train_test['train'].train_test_split(test_size = 1500)
dataset['train'] =datasets_train_validation['train']
dataset['validation'] = datasets_train_validation['test']
dataset['test']=datasets_train_test['test']

percent = 1
dataset['train'] =dataset['train'].shuffle().select(range(int(len(dataset['train']) * percent)))
dataset['validation'] =dataset['validation'].shuffle().select(range(int(len(dataset['validation']) * percent)))
dataset['test'] =dataset['test'].shuffle().select(range(int(len(dataset['test']) * percent)))
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features_finetune,  batched=True)
print(tokenized_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer',]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)
torch.save(train_dataset, 'train_data_finetuned.pt')
torch.save(valid_dataset, 'valid_data_finetuned.pt')
# logger = logging.getLogger(__name__)

training_args = TrainingArguments(output_dir="models/",
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  gradient_accumulation_steps=2,
                                  learning_rate=1e-4,
                                  num_train_epochs=7,
                                  logging_steps=100,
                                  run_name="end2end-questions-generation",
                                  evaluation_strategy="steps",
                                  save_steps=500,
                                  # report_to="wandb",
                                  push_to_hub=False,
                                  push_to_hub_model_id="flan-t5-base-samsum")

# Initialize our Trainer
trainer = Trainer(
    model=model_finetune,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)

# Training
trainer.train()
# trainer.evaluate(eval_dataset = tokenized_dataset['test'])

output_finetune_percent_trained = []
for line_idx in range(2000):
  if line_idx %100 == 0:
    print(line_idx)
  line = l[line_idx]
  qns = line['question']
  op = run_model_fine_tune(qns)
  output_finetune_percent_trained.append({'output': op, 'actual_explanation': line['full_answer'], 'actual_answer': line['exact_answer']})
print(output_finetune_percent_trained[0:10])

with open('output_finetune_percent_trained.jsonl', 'w') as outfile:
  for entry in output_finetune_percent_trained:
    json.dump(entry, outfile)
    outfile.write('\n')

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer'],
        num_rows: 7473
    })
})


Map:   0%|          | 0/4473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 4473
    })
    validation: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['question', 'full_answer', 'exact_answer', 'rationale', 'predicted_answer', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 1500
    })
})


Step,Training Loss,Validation Loss
100,1.901300,1.303532
200,1.518600,1.250628
300,1.443700,1.223124
400,1.406600,1.212727


0
100
200
300
400
500
600
700
800
900
1000


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 101>:106                                                                          │
│ in run_model_fine_tune:15                                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1611 in generate        │
│                                                                                                  │
│   1608 │   │   │   │   **model_kwargs,                                                           │
│   1609 │   │   │   )                                                                             │
│   1610 │   │   │   # 13. run beam search                                                         │
│ ❱ 1611 │   │   │   return self.beam_search(                                                      │
│   1612 │   │   │   │   input_ids,                                                                │
│   1613 │   │   │   │   beam_scorer,                                                              │
│   1614 │   │   │   │   logits_processor=logits_processor,                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2909 in beam_search     │
│                                                                                                  │
│   2906 │   │   │                                                                                 │
│   2907 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2908 │   │   │                                                                                 │
│ ❱ 2909 │   │   │   outputs = self(                                                               │
│   2910 │   │   │   │   **model_inputs,                                                           │
│   2911 │   │   │   │   return_dict=True,                                                         │
│   2912 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

In [21]:
print(output_finetune_percent_trained[0:10])

[{'output': [['Natalia sold 48 x 2 = 48*2=48>>48 clips in April. Hence, she sold a total of 48 + 48 = 240 clips. #### 240 - 240 = 224 ## 224= 240-240=3>>3 ## 3 ## 34 224+3=34*3=12>>12 ## 12 ## 13 ## 19 ## 10 ## 11 ## 23 ## 24 ## 25 ## 20 ## 22 ## 26 ## 21 ## 28 ## 27 ## 29 ## 2 ## 33 ## 31 ## 38 ## 32 ## 35 ## 30 ## 41 ## 40 ## 39 ## 45 ## 42 ## 50 ## 37 ## 44 ## 47 ## 46 ## 52 ## 54 ## 55 ## 51 ## 4 ## 49 ##']], 'actual_explanation': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'actual_answer': '72'}, {'output': [['Weng earned $12 x 60 = $12*60=60>>60 minutes of babysitting. #### 60 / 50 = 60/50=30>>30 minutes of babies were born. Therefore, she earned a total of 60 + 30 = $70 / 30 = $170+30=$70>>70 a minute of babies being babies. Thus, she earns 70 x 50 = $ 70*50=135>>135 ## 135 - 50= 135+50=245>>245 ## 245=245=175>>375 ## 375=375+445=32>>3275 $ ## 3275=$3275+32=3275> 6275= $32375+3275=1275>275+12

In [22]:
with open('output_finetune_percent_trained.jsonl', 'w') as outfile:
  for entry in output_finetune_percent_trained:
    json.dump(entry, outfile)
    outfile.write('\n')